In [ ]:

# ======================= CHECK MODEL =======================
class CHECK:
    def __init__(self, llm, tok, device):
        self.llm = llm
        self.tok = tok
        self.device = device
        self.edits = []
        self.embeds = []

        self.contriever = AutoModel.from_pretrained("facebook/contriever-msmarco").to(device)
        self.c_tok = AutoTokenizer.from_pretrained("facebook/contriever-msmarco")

    def add_edits(self, edits):
        self.edits = edits
        sr = [f"{s} {r}" for s,r,_ in edits]
        self.embeds = get_sent_embeddings(sr, self.contriever, self.c_tok, self.device)

    def llm_call(self, prompt):
        inp = self.tok(prompt, return_tensors="pt").to(self.device)
        out = self.llm.generate(**inp, max_new_tokens=32, do_sample=False)
        return self.tok.decode(out[0], skip_special_tokens=True)[len(prompt):].strip()

    def check_edit(self, s, r):
        q = get_sent_embeddings([f"{s} {r}"], self.contriever, self.c_tok, self.device)
        best, score = None, 0
        for i,e in enumerate(self.embeds):
            sim = cosine_sim(q, e.unsqueeze(0)).item()
            if sim > score and sim > 0.75:
                best = self.edits[i][2]
                score = sim
        return best

    def answer(self, question):
        s = extract_entity(question)
        if not s: return None

        rels = self.llm_call(f"Extract relations from question:\n{question}\nRelations:")
        rels = [r.strip() for r in rels.split("|") if r.strip()]

        for r in rels:
            edited = self.check_edit(s, r)
            if edited:
                s = edited
                continue
            q = self.llm_call(f"Convert to question:\n{s} {r}\nQ:")
            s = self.llm_call(f"Answer factually:\n{q}\nA:")
        return s


In [ ]:
# ======================= LOAD LLM =======================
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    device_map="auto",
    offload_folder="./offload",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

checker = CHECK(model, tokenizer, DEVICE)
len(checker.model)
plt.rcParams['figure.figsize'] = [10, 5]
checker


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:

# ======================= EVALUATION =======================
case_correct = 0
q_correct = 0
q_total = 0

for case in DATA:
    edits = []
    for rw in case["requested_rewrite"]:
        edits.append((rw["subject"], rw["relation_id"], rw["target_new"]["str"]))
    checker.add_edits(edits)

    hit = False
    for q in case["questions"]:
        q_total += 1
        pred = checker.answer(q)
        gold = case["new_answer"]
        golds = [gold["answer"]] if isinstance(gold, dict) else [gold]

        if pred and any(g.lower() in pred.lower() for g in golds):
            q_correct += 1
            hit = True

    if hit:
        case_correct += 1

print("\nFINAL RESULTS")
print("Per-case accuracy:", case_correct / len(DATA))
print("Per-question accuracy:", q_correct / q_total)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_tok

KeyboardInterrupt: 

In [ ]:
query_size == batch_size is the most important condition for the model